# 🔍 LangSmith: O "Detective Sherlock Holmes" das suas Aplicações LangChain!

## Módulo 15 - Curso LangChain v0.2

E aí, pessoal! Pedro Guth aqui! 🚀

Tá, depois de construirmos projetos incríveis com LangChain e LangGraph, chegou a hora de responder uma pergunta crucial: **"Como diabos eu monitoro e otimizo essas aplicações em produção?"**

É aí que entra o **LangSmith** - pense nele como o Sherlock Holmes das suas aplicações de IA! 🕵️‍♂️

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_01.png)

### O que você vai aprender hoje:
- 🔍 O que é LangSmith e por que você PRECISA dele
- 📊 Como fazer observabilidade completa das suas chains
- 🐛 Debug avançado de aplicações LangChain
- 📈 Métricas e análises que importam
- 💡 Otimização baseada em dados reais

Bora descobrir os segredos por trás das suas aplicações! 🔍

## 🤔 Tá, mas o que é LangSmith?

Imagina que você é um chef de um restaurante super movimentado. Você criou receitas incríveis (suas chains do LangChain), mas agora precisa saber:

- **Qual prato está demorando mais para ficar pronto?** (Latência das chains)
- **Onde os garçons estão errando os pedidos?** (Onde suas chains falham)
- **Qual ingrediente está custando mais caro?** (Custo dos tokens)
- **Os clientes estão satisfeitos?** (Qualidade das respostas)

**LangSmith é como ter câmeras, cronômetros e um sistema de qualidade em toda sua cozinha!** 👨‍🍳

### Principais recursos:
1. **Tracing**: Rastreia cada passo das suas chains
2. **Evaluation**: Avalia qualidade das respostas
3. **Monitoring**: Monitora performance em tempo real
4. **Debugging**: Identifica gargalos e erros
5. **Dataset Management**: Gerencia dados de teste



In [ ]:
# Primeiro, vamos instalar as dependências necessárias
!pip install langsmith langchain langchain-google-genai python-dotenv

# Imports necessários
import os
from dotenv import load_dotenv
import datetime
from typing import List, Dict, Any

# LangChain imports (já conhecemos do curso!)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# LangSmith imports - A novidade!
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Dataset, Example

print("📦 Dependências instaladas com sucesso!")
print("🎯 Vamos explorar o LangSmith!")

In [ ]:
# Configuração das variáveis de ambiente
load_dotenv()

# Configuração do Google AI (já conhecemos!)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("🔑 Digite sua Google API Key: ")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Configuração do LangSmith - NOVO!
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
if not LANGCHAIN_API_KEY:
    print("\n🆕 Para usar LangSmith, você precisa de uma conta em https://smith.langchain.com")
    LANGCHAIN_API_KEY = input("🔑 Digite sua LangChain API Key (ou ENTER para pular): ")
    if LANGCHAIN_API_KEY:
        os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

# Configurações do LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Ativa o tracing
os.environ["LANGCHAIN_PROJECT"] = "langchain-course-module-15"  # Nome do projeto

print("✅ Configuração concluída!")
print(f"📊 Projeto LangSmith: {os.environ.get('LANGCHAIN_PROJECT', 'Não configurado')}")

## 🏗️ Criando uma Chain para Monitorar

Vamos criar uma chain simples que já usamos no curso - um assistente que explica conceitos técnicos. Mas agora, com LangSmith, vamos poder ver **tudo** que acontece por debaixo dos panos!

**Dica do Pedro**: É como ter raio-X da sua aplicação! 🩻

In [ ]:
# Criando nosso modelo (já conhecemos do módulo 2!)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    max_tokens=1000
)

# Prompt template (módulo 3!)
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um professor de tecnologia brasileiro, didático e descontraído.
    Explique conceitos técnicos de forma simples, usando analogias do cotidiano.
    Seja informal mas preciso.
    """),
    ("user", "Explique o conceito: {conceito}")
])

# Parser de saída (módulo 3!)
parser = StrOutputParser()

# Nossa chain completa (módulo 4!)
explicador_chain = prompt | llm | parser

print("🔗 Chain criada com sucesso!")
print("📡 Agora com monitoramento LangSmith ativo!")

## 🚀 Primeira Execução com Tracing

Tá, agora vem a mágica! Quando executarmos nossa chain, o LangSmith vai **automaticamente** capturar:

- ⏱️ **Tempo de execução** de cada etapa
- 💰 **Tokens consumidos** (entrada e saída)
- 🔄 **Fluxo completo** da execução
- ❌ **Erros** (se houver)
- 📊 **Metadados** diversos

É como ter um **GPS** mostrando todo o trajeto da sua chain! 🗺️

In [ ]:
# Vamos testar nossa chain com diferentes conceitos
conceitos_teste = [
    "Machine Learning",
    "API REST",
    "Docker",
    "Microserviços"
]

print("🔍 Executando chains com monitoramento LangSmith...\n")

resultados = []

for i, conceito in enumerate(conceitos_teste, 1):
    print(f"📚 {i}/4 - Explicando: {conceito}")
    
    # Aqui a mágica acontece! LangSmith captura tudo automaticamente
    try:
        resultado = explicador_chain.invoke({"conceito": conceito})
        resultados.append({
            "conceito": conceito,
            "explicacao": resultado[:200] + "...",  # Primeiros 200 chars
            "status": "✅ Sucesso"
        })
        print(f"   {resultado[:100]}...")
        
    except Exception as e:
        resultados.append({
            "conceito": conceito,
            "explicacao": f"Erro: {str(e)}",
            "status": "❌ Erro"
        })
        print(f"   ❌ Erro: {e}")
    
    print("   📊 Trace enviado para LangSmith!\n")

print("🎉 Todas as execuções concluídas!")
print("🔗 Acesse https://smith.langchain.com para ver os traces!")

## 📊 Visualizando Métricas Básicas

Agora vamos criar algumas visualizações básicas para entender o que está acontecendo. É como fazer um **raio-X** das nossas execuções!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Simulando dados de performance (em produção, você pegaria do LangSmith)
conceitos = [r['conceito'] for r in resultados]
tempos_execucao = np.random.uniform(0.5, 3.0, len(conceitos))  # Simulando tempos
tokens_usados = np.random.randint(100, 500, len(conceitos))    # Simulando tokens

# Gráfico de tempos de execução
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tempo de execução
bars1 = ax1.bar(range(len(conceitos)), tempos_execucao, 
                color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax1.set_xlabel('Conceitos Explicados')
ax1.set_ylabel('Tempo (segundos)')
ax1.set_title('⏱️ Tempo de Execução por Conceito', fontsize=14, fontweight='bold')
ax1.set_xticks(range(len(conceitos)))
ax1.set_xticklabels([c[:10]+'...' if len(c) > 10 else c for c in conceitos], rotation=45)

# Adicionando valores nas barras
for bar, tempo in zip(bars1, tempos_execucao):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{tempo:.1f}s', ha='center', va='bottom')

# Gráfico 2: Tokens utilizados
bars2 = ax2.bar(range(len(conceitos)), tokens_usados,
                color=['#FFB6C1', '#98FB98', '#87CEEB', '#DDA0DD'])
ax2.set_xlabel('Conceitos Explicados')
ax2.set_ylabel('Tokens Utilizados')
ax2.set_title('🪙 Tokens Consumidos por Conceito', fontsize=14, fontweight='bold')
ax2.set_xticks(range(len(conceitos)))
ax2.set_xticklabels([c[:10]+'...' if len(c) > 10 else c for c in conceitos], rotation=45)

# Adicionando valores nas barras
for bar, tokens in zip(bars2, tokens_usados):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{tokens}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("📊 Métricas simuladas geradas!")
print("💡 Em produção, esses dados viriam diretamente do LangSmith!")

## 🔧 Client LangSmith: Acessando Dados Programaticamente

Tá, agora vamos ver como acessar os dados do LangSmith de forma programática. É como ter **acesso VIP** aos bastidores! 🎭

**Dica do Pedro**: Com o cliente, você pode criar dashboards personalizados, alertas automáticos e muito mais!

In [ ]:
# Inicializando o cliente LangSmith
try:
    client = Client()
    print("✅ Cliente LangSmith conectado com sucesso!")
    
    # Verificando informações do projeto atual
    project_name = os.environ.get("LANGCHAIN_PROJECT", "default")
    print(f"📊 Projeto atual: {project_name}")
    
    # Função para listar execuções recentes
    def listar_execucoes_recentes(limit=5):
        """Lista as execuções mais recentes do projeto"""
        try:
            runs = client.list_runs(
                project_name=project_name,
                limit=limit
            )
            
            print(f"\n🔍 Últimas {limit} execuções:")
            print("-" * 80)
            
            for i, run in enumerate(runs, 1):
                status_icon = "✅" if run.status == "success" else "❌"
                print(f"{i}. {status_icon} {run.name} | ID: {str(run.id)[:8]}...")
                if hasattr(run, 'start_time') and run.start_time:
                    print(f"   📅 Executado em: {run.start_time.strftime('%H:%M:%S')}")
                if hasattr(run, 'execution_order'):
                    print(f"   ⚡ Ordem: {run.execution_order}")
                print()
                
        except Exception as e:
            print(f"⚠️ Não foi possível listar execuções: {e}")
            print("💡 Verifique se você tem execuções no projeto atual")
    
    # Listando execuções
    listar_execucoes_recentes()
    
except Exception as e:
    print(f"❌ Erro ao conectar com LangSmith: {e}")
    print("💡 Verifique se sua LANGCHAIN_API_KEY está configurada corretamente")
    client = None

## 📈 Arquitetura do LangSmith

Vamos entender como o LangSmith funciona por baixo dos panos. É como conhecer a **arquitetura** de uma cidade inteligente! 🏙️

In [ ]:
from IPython.display import display, HTML

# Diagrama da arquitetura LangSmith
mermaid_diagram = """
```mermaid
graph TD
    A[🔗 Sua Aplicação LangChain] --> B[📡 LangSmith Tracer]
    B --> C[☁️ LangSmith Cloud]
    
    C --> D[📊 Dashboard Web]
    C --> E[🔍 API Client]
    C --> F[📈 Analytics Engine]
    
    D --> G[👁️ Visualização de Traces]
    D --> H[📋 Relatórios]
    
    E --> I[🤖 Automações]
    E --> J[🚨 Alertas]
    
    F --> K[📊 Métricas]
    F --> L[🎯 Insights]
    
    style A fill:#e1f5fe
    style C fill:#f3e5f5
    style D fill:#e8f5e8
```
"""

print("🏗️ Arquitetura do LangSmith:")
print(mermaid_diagram)

print("\n📋 Componentes principais:")
print("1. 📡 Tracer: Captura automaticamente todas as execuções")
print("2. ☁️ Cloud: Armazena e processa os dados")
print("3. 📊 Dashboard: Interface visual para análise")
print("4. 🔍 API: Acesso programático aos dados")
print("5. 📈 Analytics: Processamento e insights automáticos")

## 🧪 Evaluation: Testando Qualidade

Uma das funcionalidades mais **poderosas** do LangSmith é a capacidade de avaliar a qualidade das suas chains automaticamente!

É como ter um **corretor automático** que nunca erra! 📝

**Dica do Pedro**: Em produção, você pode configurar avaliações automáticas para cada deploy!

In [ ]:
# Criando um dataset de teste
def criar_dataset_teste():
    """Cria um dataset para avaliar nossa chain explicadora"""
    
    exemplos_teste = [
        {
            "conceito": "Inteligência Artificial",
            "explicacao_esperada": "Sistema que simula inteligência humana"
        },
        {
            "conceito": "Blockchain",
            "explicacao_esperada": "Tecnologia de registro distribuído e imutável"
        },
        {
            "conceito": "Cloud Computing",
            "explicacao_esperada": "Computação usando recursos remotos via internet"
        }
    ]
    
    return exemplos_teste

# Função de avaliação personalizada
def avaliar_qualidade_explicacao(run, example):
    """Avalia se a explicação contém elementos essenciais"""
    
    explicacao = run.outputs.get("output", "").lower()
    conceito = example.inputs.get("conceito", "").lower()
    
    # Critérios simples de avaliação
    pontuacao = 0
    feedback = []
    
    # 1. Menciona o conceito?
    if any(palavra in explicacao for palavra in conceito.split()):
        pontuacao += 0.3
        feedback.append("✅ Menciona o conceito")
    else:
        feedback.append("❌ Não menciona claramente o conceito")
    
    # 2. Tem analogia ou exemplo?
    palavras_analogia = ["como", "igual", "assim", "exemplo", "imagina", "pense"]
    if any(palavra in explicacao for palavra in palavras_analogia):
        pontuacao += 0.3
        feedback.append("✅ Usa analogias/exemplos")
    else:
        feedback.append("⚠️ Poderia usar mais analogias")
    
    # 3. Tamanho adequado?
    if 50 <= len(explicacao) <= 500:
        pontuacao += 0.2
        feedback.append("✅ Tamanho adequado")
    else:
        feedback.append("⚠️ Tamanho inadequado")
    
    # 4. Tom informal?
    palavras_informais = ["né", "tá", "galera", "pessoal", "cara", "mano"]
    if any(palavra in explicacao for palavra in palavras_informais):
        pontuacao += 0.2
        feedback.append("✅ Tom informal")
    else:
        feedback.append("⚠️ Poderia ser mais informal")
    
    return {
        "key": "qualidade_explicacao",
        "score": pontuacao,
        "comment": " | ".join(feedback)
    }

# Testando nossa função de avaliação
exemplos = criar_dataset_teste()
print("🧪 Dataset de teste criado!")
print(f"📊 {len(exemplos)} exemplos preparados para avaliação")

# Simulando uma avaliação
print("\n🔬 Exemplo de avaliação:")
class MockRun:
    def __init__(self, output):
        self.outputs = {"output": output}

class MockExample:
    def __init__(self, conceito):
        self.inputs = {"conceito": conceito}

# Testando com uma explicação simulada
mock_run = MockRun("Inteligência Artificial é como ter um robô super inteligente que pensa como a gente, né? É tecnologia que simula o cérebro humano.")
mock_example = MockExample("Inteligência Artificial")

resultado_avaliacao = avaliar_qualidade_explicacao(mock_run, mock_example)
print(f"📊 Pontuação: {resultado_avaliacao['score']:.1f}/1.0")
print(f"💬 Feedback: {resultado_avaliacao['comment']}")

## 🔍 Debugging Avançado

Uma das coisas mais **Liiiindas** do LangSmith é a capacidade de fazer debug detalhado das suas chains!

É como ter **lupa de detective** para encontrar bugs! 🔍

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_03.png)

In [ ]:
# Vamos criar uma chain mais complexa para demonstrar debugging
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda

# Chain com múltiplas etapas (como vimos no módulo 4!)
def extrair_palavras_chave(conceito: str) -> dict:
    """Extrai palavras-chave do conceito para análise"""
    palavras = conceito.lower().split()
    
    # Simulando algum processamento
    import time
    time.sleep(0.1)  # Simula processamento
    
    return {
        "conceito_original": conceito,
        "palavras_chave": palavras,
        "num_palavras": len(palavras),
        "complexidade": "alta" if len(palavras) > 2 else "baixa"
    }

def personalizar_prompt(dados: dict) -> dict:
    """Personaliza o prompt baseado na complexidade"""
    conceito = dados["conceito_original"]
    complexidade = dados["complexidade"]
    
    if complexidade == "alta":
        estilo = "Esse conceito é complexo, então seja bem didático e use várias analogias."
    else:
        estilo = "Conceito simples, pode ser mais direto mas ainda informal."
    
    return {
        "conceito": conceito,
        "estilo_instrucao": estilo,
        **dados  # Mantém dados originais
    }

# Prompt dinâmico
prompt_avancado = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um professor de tecnologia brasileiro, didático e descontraído.
    {estilo_instrucao}
    Explique conceitos técnicos de forma simples, usando analogias do cotidiano.
    Seja informal mas preciso.
    """),
    ("user", "Explique o conceito: {conceito}")
])

# Chain completa com debugging
chain_com_debug = (
    RunnableLambda(extrair_palavras_chave) |
    RunnableLambda(personalizar_prompt) |
    prompt_avancado |
    llm |
    parser
)

print("🔧 Chain avançada criada com múltiplas etapas!")
print("🔍 Cada etapa será trackeada pelo LangSmith!")

In [ ]:
# Testando a chain avançada
conceitos_debug = [
    "API",  # Simples
    "Machine Learning Ops",  # Complexo
    "GraphQL"  # Médio
]

print("🔍 Testando chain avançada com debugging detalhado...\n")

for conceito in conceitos_debug:
    print(f"🧠 Processando: {conceito}")
    
    try:
        # Execução com tracing automático
        resultado = chain_com_debug.invoke(conceito)
        
        print(f"   ✅ Processado com sucesso!")
        print(f"   📝 Resposta: {resultado[:100]}...")
        print(f"   📊 Trace completo enviado para LangSmith!")
        
    except Exception as e:
        print(f"   ❌ Erro durante processamento: {e}")
        print(f"   🔍 Erro também trackeado no LangSmith!")
    
    print("-" * 50)

print("\n🎉 Debugging completo!")
print("💡 Agora você pode ver no LangSmith:")
print("   - Tempo de cada etapa da chain")
print("   - Input/Output de cada função")
print("   - Erros detalhados (se houver)")
print("   - Fluxo completo de execução")

## 📊 Análise de Performance

Vamos criar um dashboard simples para analisar a performance das nossas chains. É como ter um **painel de controle** da sua aplicação! 🎛️

**Dica do Pedro**: Esses tipos de análise são essenciais para otimizar custos e performance em produção!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

# Simulando dados de performance ao longo do tempo
# Em produção, esses dados viriam do LangSmith API

# Gerando dados simulados para uma semana
dias = 7
horas_por_dia = 24
total_pontos = dias * horas_por_dia

# Timestamps
agora = datetime.now()
timestamps = [agora - timedelta(hours=i) for i in range(total_pontos, 0, -1)]

# Métricas simuladas
np.random.seed(42)  # Para resultados consistentes

# Latência (com padrão diário)
latencia_base = 1.5
latencia = []
for i, ts in enumerate(timestamps):
    # Maior latência durante horário comercial (9-18h)
    hora = ts.hour
    if 9 <= hora <= 18:
        fator_hora = 1.3  # 30% mais lento
    else:
        fator_hora = 0.8  # 20% mais rápido
    
    # Adiciona ruído aleatório
    ruido = np.random.normal(0, 0.2)
    lat = latencia_base * fator_hora + ruido
    latencia.append(max(0.1, lat))  # Mínimo 0.1s

# Taxa de sucesso
taxa_sucesso = np.random.uniform(0.92, 0.99, total_pontos)

# Custo (tokens)
custo_por_hora = np.random.poisson(150, total_pontos)  # Média 150 tokens/hora

print(f"📊 Dados simulados gerados para {dias} dias")
print(f"⏱️ Latência média: {np.mean(latencia):.2f}s")
print(f"✅ Taxa de sucesso média: {np.mean(taxa_sucesso):.1%}")
print(f"💰 Tokens médios/hora: {np.mean(custo_por_hora):.0f}")

In [ ]:
# Dashboard de Performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎛️ Dashboard de Performance - LangSmith Analytics', fontsize=16, fontweight='bold')

# 1. Latência ao longo do tempo
ax1.plot(timestamps, latencia, color='#FF6B6B', linewidth=1.5, alpha=0.8)
ax1.fill_between(timestamps, latencia, alpha=0.3, color='#FF6B6B')
ax1.set_title('⏱️ Latência das Execuções', fontweight='bold')
ax1.set_ylabel('Tempo (segundos)')
ax1.grid(True, alpha=0.3)

# Linha de média
media_latencia = np.mean(latencia)
ax1.axhline(y=media_latencia, color='red', linestyle='--', alpha=0.8, 
           label=f'Média: {media_latencia:.2f}s')
ax1.legend()

# 2. Taxa de sucesso
# Agrupando por dia para melhor visualização
dias_unicos = list(set(ts.date() for ts in timestamps))
dias_unicos.sort()

taxa_por_dia = []
for dia in dias_unicos:
    indices_dia = [i for i, ts in enumerate(timestamps) if ts.date() == dia]
    taxa_dia = np.mean([taxa_sucesso[i] for i in indices_dia])
    taxa_por_dia.append(taxa_dia)

bars = ax2.bar(range(len(dias_unicos)), [t*100 for t in taxa_por_dia], 
               color='#4ECDC4', alpha=0.8)
ax2.set_title('✅ Taxa de Sucesso por Dia', fontweight='bold')
ax2.set_ylabel('Taxa de Sucesso (%)')
ax2.set_ylim(90, 100)
ax2.set_xticks(range(len(dias_unicos)))
ax2.set_xticklabels([dia.strftime('%d/%m') for dia in dias_unicos], rotation=45)

# Adicionando valores nas barras
for bar, taxa in zip(bars, taxa_por_dia):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{taxa:.1%}', ha='center', va='bottom', fontweight='bold')

# 3. Distribuição de latência (histograma)
ax3.hist(latencia, bins=30, color='#45B7D1', alpha=0.7, edgecolor='black')
ax3.set_title('📊 Distribuição de Latência', fontweight='bold')
ax3.set_xlabel('Tempo (segundos)')
ax3.set_ylabel('Frequência')
ax3.axvline(x=media_latencia, color='red', linestyle='--', 
           label=f'Média: {media_latencia:.2f}s')
ax3.axvline(x=np.percentile(latencia, 95), color='orange', linestyle='--', 
           label=f'P95: {np.percentile(latencia, 95):.2f}s')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Custo acumulado (tokens)
custo_acumulado = np.cumsum(custo_por_hora)
ax4.plot(timestamps, custo_acumulado, color='#96CEB4', linewidth=2)
ax4.fill_between(timestamps, custo_acumulado, alpha=0.3, color='#96CEB4')
ax4.set_title('💰 Tokens Acumulados', fontweight='bold')
ax4.set_ylabel('Total de Tokens')
ax4.grid(True, alpha=0.3)

# Formatação dos eixos X para mostrar datas
for ax in [ax1, ax4]:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Resumo das métricas
print("\n📈 RESUMO DAS MÉTRICAS:")
print("=" * 50)
print(f"⏱️  Latência Média: {np.mean(latencia):.2f}s")
print(f"🚀 Latência P95: {np.percentile(latencia, 95):.2f}s")
print(f"✅ Taxa de Sucesso: {np.mean(taxa_sucesso):.1%}")
print(f"💰 Total de Tokens: {custo_acumulado[-1]:,}")
print(f"📊 Execuções Simuladas: {len(latencia)}")

## 🎯 Casos de Uso Reais do LangSmith

Tá, agora vamos ver onde o LangSmith **realmente brilha** no mundo real! São cenários que você vai enfrentar quando colocar suas aplicações em produção.

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_04.png)

## 💡 Cenários Práticos

### 1. 🏥 Sistema de Atendimento Médico
**Problema**: Um hospital usa LangChain para classificar sintomas dos pacientes
**LangSmith ajuda**:
- Monitora precisão das classificações
- Detecta quando modelo está errando muito
- Rastreia tempo de resposta (crítico em emergências)
- Avalia qualidade das respostas médicas

### 2. 🛒 E-commerce com Chatbot
**Problema**: Loja online com assistente virtual para vendas
**LangSmith ajuda**:
- Analisa quais produtos geram mais dúvidas
- Monitora taxa de conversão por conversa
- Detecta quando chatbot não está ajudando
- Otimiza prompts baseado em dados reais

### 3. 📚 Plataforma Educacional
**Problema**: Sistema que gera explicações personalizadas
**LangSmith ajuda**:
- Avalia didática das explicações
- Monitora engagement dos alunos
- Detecta tópicos mais difíceis
- A/B testing de diferentes approaches

**Dica do Pedro**: Em cada caso, LangSmith transforma dados em insights acionáveis! 📊

In [ ]:
# Simulando um sistema de alertas inteligentes
# Isso é algo que você faria em produção com LangSmith

class SistemaAlertas:
    def __init__(self):
        self.thresholds = {
            'latencia_max': 3.0,      # 3 segundos
            'taxa_sucesso_min': 0.95,  # 95%
            'custo_hora_max': 1000,     # 1000 tokens/hora
            'taxa_erro_max': 0.05       # 5%
        }
        self.alertas_ativos = []
    
    def verificar_metricas(self, metricas):
        """Verifica se alguma métrica ultrapassou o threshold"""
        alertas = []
        
        # Verifica latência
        if metricas['latencia_media'] > self.thresholds['latencia_max']:
            alertas.append({
                'tipo': '🐌 LATÊNCIA ALTA',
                'valor': f"{metricas['latencia_media']:.2f}s",
                'threshold': f"{self.thresholds['latencia_max']}s",
                'severidade': 'ALTA' if metricas['latencia_media'] > 5.0 else 'MÉDIA'
            })
        
        # Verifica taxa de sucesso
        if metricas['taxa_sucesso'] < self.thresholds['taxa_sucesso_min']:
            alertas.append({
                'tipo': '❌ TAXA DE ERRO ALTA',
                'valor': f"{metricas['taxa_sucesso']:.1%}",
                'threshold': f"{self.thresholds['taxa_sucesso_min']:.1%}",
                'severidade': 'CRÍTICA' if metricas['taxa_sucesso'] < 0.90 else 'ALTA'
            })
        
        # Verifica custo
        if metricas['tokens_por_hora'] > self.thresholds['custo_hora_max']:
            alertas.append({
                'tipo': '💸 CUSTO ALTO',
                'valor': f"{metricas['tokens_por_hora']:,} tokens/hora",
                'threshold': f"{self.thresholds['custo_hora_max']:,} tokens/hora",
                'severidade': 'MÉDIA'
            })
        
        return alertas
    
    def gerar_relatorio_alertas(self, alertas):
        """Gera relatório formatado dos alertas"""
        if not alertas:
            return "✅ Todas as métricas estão dentro dos parâmetros normais!"
        
        relatorio = "🚨 ALERTAS DETECTADOS:\n"
        relatorio += "=" * 60 + "\n"
        
        for alerta in alertas:
            relatorio += f"\n{alerta['tipo']} - {alerta['severidade']}\n"
            relatorio += f"   📊 Valor atual: {alerta['valor']}\n"
            relatorio += f"   🎯 Threshold: {alerta['threshold']}\n"
            relatorio += "-" * 40 + "\n"
        
        return relatorio

# Testando o sistema de alertas
sistema_alertas = SistemaAlertas()

# Cenário 1: Métricas normais
metricas_normais = {
    'latencia_media': 1.2,
    'taxa_sucesso': 0.98,
    'tokens_por_hora': 200
}

print("📊 CENÁRIO 1 - Operação Normal:")
alertas1 = sistema_alertas.verificar_metricas(metricas_normais)
print(sistema_alertas.gerar_relatorio_alertas(alertas1))

print("\n" + "="*60 + "\n")

# Cenário 2: Problemas detectados
metricas_problemas = {
    'latencia_media': 4.5,
    'taxa_sucesso': 0.89,
    'tokens_por_hora': 1500
}

print("📊 CENÁRIO 2 - Problemas Detectados:")
alertas2 = sistema_alertas.verificar_metricas(metricas_problemas)
print(sistema_alertas.gerar_relatorio_alertas(alertas2))

## 🏋️‍♀️ Exercício Prático 1: Criando seu Sistema de Monitoramento

Bora botar a mão na massa! Você vai criar uma chain personalizada e implementar seu próprio sistema de monitoramento.

**Desafio**: Implemente uma chain que classifica sentimentos de reviews e monitore sua performance!

**Dica do Pedro**: Pense como um DevOps - quais métricas são realmente importantes? 🤔

In [ ]:
# EXERCÍCIO 1: Sistema de Análise de Sentimentos com Monitoramento
# Complete o código abaixo!

# 1. Crie um prompt para análise de sentimentos
prompt_sentimentos = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um especialista em análise de sentimentos.
    Classifique o sentimento do texto em: POSITIVO, NEGATIVO ou NEUTRO.
    Seja preciso e consistente.
    Responda apenas com a classificação seguida de um score de 0-10.
    Formato: "POSITIVO - 8" ou "NEGATIVO - 3"
    """),
    ("user", "Analise o sentimento deste texto: {texto}")
])

# 2. Crie a chain de análise
# SEU CÓDIGO AQUI:
chain_sentimentos = None  # Substitua por sua implementação

# 3. Dataset de teste
reviews_teste = [
    "Produto excelente! Superou todas as expectativas",
    "Terrível, não funcionou nem um dia",
    "Ok, cumpre o que promete",
    "Incrível! Recomendo para todos",
    "Péssimo atendimento, produto com defeito",
    "Funciona bem, mas poderia ser melhor"
]

# 4. Implemente uma função de avaliação
def avaliar_classificacao_sentimento(resultado, texto_original):
    """Avalia a qualidade da classificação de sentimento"""
    # SEU CÓDIGO AQUI:
    # Dicas:
    # - Verifique se o formato está correto
    # - Valide se a classificação faz sentido
    # - Retorne um score de 0-1
    pass

# 5. Execute os testes
print("🎯 EXERCÍCIO 1: Análise de Sentimentos")
print("Implemente o código acima e teste sua chain!")
print("\n📝 Checklist:")
print("[ ] Chain de sentimentos criada")
print("[ ] Função de avaliação implementada")
print("[ ] Testes executados com sucesso")
print("[ ] Métricas coletadas e analisadas")

## 🎓 Exercício Prático 2: Dashboard Personalizado

Agora vamos criar um dashboard personalizado para monitorar múltiplas chains em tempo real!

**Desafio**: Implemente um sistema que monitore 3 tipos diferentes de chains e gere alertas inteligentes.

**Dica do Pedro**: Pense em como diferentes tipos de aplicação precisam de métricas diferentes!

In [ ]:
# EXERCÍCIO 2: Dashboard Multi-Chain
# Complete a implementação!

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

class DashboardMultiChain:
    def __init__(self):
        self.chains_data = {
            'chat_bot': {
                'name': '🤖 ChatBot Atendimento',
                'metricas': [],
                'color': '#FF6B6B'
            },
            'classificador': {
                'name': '🏷️ Classificador Sentimentos', 
                'metricas': [],
                'color': '#4ECDC4'
            },
            'resumidor': {
                'name': '📝 Resumidor Textos',
                'metricas': [],
                'color': '#45B7D1'
            }
        }
    
    def adicionar_metrica(self, chain_name, latencia, taxa_sucesso, tokens):
        """Adiciona nova métrica para uma chain"""
        # SEU CÓDIGO AQUI:
        # Implemente a lógica para adicionar métricas
        pass
    
    def gerar_dashboard(self):
        """Gera dashboard visual com todas as chains"""
        # SEU CÓDIGO AQUI:
        # Crie gráficos comparativos entre as chains
        # Dicas:
        # - Use subplots para múltiplas visualizações
        # - Compare latência entre chains
        # - Mostre evolução temporal
        # - Destaque chains com problemas
        pass
    
    def detectar_anomalias(self):
        """Detecta anomalias nas métricas das chains"""
        # SEU CÓDIGO AQUI:
        # Implemente detecção de anomalias
        # Dicas:
        # - Use desvio padrão para detectar outliers
        # - Compare com médias históricas
        # - Gere alertas específicos por tipo de chain
        pass

# Simulação de dados para teste
dashboard = DashboardMultiChain()

# Gerando dados simulados
np.random.seed(42)
for i in range(24):  # 24 horas de dados
    # ChatBot - mais usado durante dia
    dashboard.adicionar_metrica('chat_bot', 
                               np.random.uniform(0.8, 2.5),
                               np.random.uniform(0.92, 0.98),
                               np.random.randint(200, 800))
    
    # Classificador - performance mais estável
    dashboard.adicionar_metrica('classificador',
                               np.random.uniform(0.3, 1.0),
                               np.random.uniform(0.95, 0.99),
                               np.random.randint(50, 200))
    
    # Resumidor - mais pesado
    dashboard.adicionar_metrica('resumidor',
                               np.random.uniform(2.0, 5.0),
                               np.random.uniform(0.88, 0.96),
                               np.random.randint(300, 1000))

print("📊 EXERCÍCIO 2: Dashboard Multi-Chain")
print("Complete a implementação da classe DashboardMultiChain!")
print("\n📝 Checklist:")
print("[ ] Método adicionar_metrica implementado")
print("[ ] Dashboard visual criado")
print("[ ] Sistema de detecção de anomalias")
print("[ ] Alertas específicos por tipo de chain")
print("[ ] Comparações entre chains")

## 🔮 LangSmith vs Outras Ferramentas

Tá, mas por que LangSmith e não outras opções? Vamos fazer uma **comparação honesta**! 🥊

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_05.png)

In [ ]:
# Comparação visual das ferramentas de monitoramento
from IPython.display import display, HTML

# Dados da comparação
ferramentas = {
    'LangSmith': {
        'integração_langchain': 10,
        'facilidade_uso': 9,
        'recursos_debugging': 10,
        'custo': 6,
        'personalização': 8
    },
    'Weights & Biases': {
        'integração_langchain': 7,
        'facilidade_uso': 8,
        'recursos_debugging': 7,
        'custo': 7,
        'personalização': 9
    },
    'MLflow': {
        'integração_langchain': 6,
        'facilidade_uso': 7,
        'recursos_debugging': 6,
        'custo': 10,  # Open source
        'personalização': 8
    },
    'Datadog': {
        'integração_langchain': 5,
        'facilidade_uso': 8,
        'recursos_debugging': 8,
        'custo': 4,
        'personalização': 7
    }
}

# Gráfico radar para comparação
import matplotlib.pyplot as plt
import numpy as np

# Configuração do gráfico radar
criterios = list(ferramentas['LangSmith'].keys())
num_criterios = len(criterios)

# Ângulos para cada critério
angulos = np.linspace(0, 2 * np.pi, num_criterios, endpoint=False).tolist()
angulos += angulos[:1]  # Fecha o círculo

fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(projection='polar'))

cores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, (ferramenta, dados) in enumerate(ferramentas.items()):
    valores = list(dados.values())
    valores += valores[:1]  # Fecha o círculo
    
    ax.plot(angulos, valores, 'o-', linewidth=2, 
           label=ferramenta, color=cores[i])
    ax.fill(angulos, valores, alpha=0.25, color=cores[i])

# Configurações do gráfico
ax.set_xticks(angulos[:-1])
ax.set_xticklabels([c.replace('_', ' ').title() for c in criterios])
ax.set_ylim(0, 10)
ax.set_yticks(range(0, 11, 2))
ax.set_title('🥊 Comparação de Ferramentas de Monitoramento\n(Escala 0-10)', 
             size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.grid(True)

plt.tight_layout()
plt.show()

print("🎯 ANÁLISE DAS FERRAMENTAS:")
print("=" * 50)
print("🏆 LangSmith:")
print("   ✅ Integração nativa com LangChain")
print("   ✅ Debugging específico para chains")
print("   ✅ Tracing automático")
print("   ❌ Mais caro que alternativas open-source")
print("\n🥈 Weights & Biases:")
print("   ✅ Excelente para ML em geral")
print("   ✅ Muito flexível")
print("   ❌ Curva de aprendizado maior")
print("\n🥉 MLflow:")
print("   ✅ Open source e gratuito")
print("   ✅ Comunidade ativa")
print("   ❌ Menos específico para LLMs")
print("\n💼 Datadog:")
print("   ✅ Monitoramento enterprise")
print("   ✅ Infraestrutura robusta")
print("   ❌ Muito caro")
print("   ❌ Menos foco em LLMs")

## 🚀 Próximos Passos e Melhores Práticas

Tá, agora que você já sabe **tudo** sobre LangSmith, vamos falar sobre como usar isso na prática! É hora de partir para o **próximo nível**! 🚀

**Dica do Pedro**: A diferença entre um desenvolvedor junior e senior é saber monitorar e otimizar suas aplicações!

## 📋 Checklist de Implementação em Produção

### 🎯 Antes de ir para produção:

#### 1. **Setup Básico** ✅
- [ ] Conta LangSmith configurada
- [ ] Variáveis de ambiente definidas
- [ ] Projetos organizados por ambiente (dev/staging/prod)
- [ ] API keys seguras

#### 2. **Monitoramento** 📊
- [ ] Tracing ativo em todas as chains
- [ ] Métricas de latência configuradas
- [ ] Monitoramento de custos (tokens)
- [ ] Alertas automáticos configurados

#### 3. **Qualidade** 🎯
- [ ] Datasets de avaliação criados
- [ ] Critérios de qualidade definidos
- [ ] Testes automatizados implementados
- [ ] Processo de review das respostas

#### 4. **Otimização** ⚡
- [ ] Dashboards de performance
- [ ] Análise de gargalos
- [ ] A/B testing de prompts
- [ ] Otimização baseada em dados

### 🔥 Dicas Avançadas:

1. **Organize por contexto**: Crie projetos separados para cada aplicação
2. **Tags inteligentes**: Use tags para categorizar execuções
3. **Filtros poderosos**: Configure filtros para análises específicas
4. **Relatórios automáticos**: Configure reports semanais
5. **Integração CI/CD**: Inclua avaliações no seu pipeline

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_06.png)

In [ ]:
# Template de configuração para produção
# Use isso como base para seus projetos!

import os
from datetime import datetime

class ConfigLangSmithProd:
    """Configuração completa do LangSmith para produção"""
    
    def __init__(self, ambiente="dev"):
        self.ambiente = ambiente
        self.configurar_ambiente()
    
    def configurar_ambiente(self):
        """Configura variáveis baseadas no ambiente"""
        
        # Configurações base
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
        
        # Configurações por ambiente
        if self.ambiente == "dev":
            os.environ["LANGCHAIN_PROJECT"] = f"meu-projeto-dev-{datetime.now().strftime('%Y%m')}"
            self.sample_rate = 1.0  # 100% das execuções
            
        elif self.ambiente == "staging":
            os.environ["LANGCHAIN_PROJECT"] = f"meu-projeto-staging-{datetime.now().strftime('%Y%m')}"
            self.sample_rate = 0.5  # 50% das execuções
            
        elif self.ambiente == "prod":
            os.environ["LANGCHAIN_PROJECT"] = f"meu-projeto-prod-{datetime.now().strftime('%Y%m')}"
            self.sample_rate = 0.1  # 10% das execuções (para reduzir custos)
        
        print(f"🚀 LangSmith configurado para {self.ambiente}")
        print(f"📊 Projeto: {os.environ.get('LANGCHAIN_PROJECT')}")
        print(f"📈 Sample rate: {self.sample_rate*100}%")
    
    def configurar_tags_padrao(self, chain_name, versao="1.0"):
        """Define tags padrão para organização"""
        tags = {
            "ambiente": self.ambiente,
            "chain_name": chain_name,
            "versao": versao,
            "timestamp": datetime.now().isoformat()
        }
        return tags
    
    def health_check(self):
        """Verifica se a configuração está ok"""
        checks = {
            "LANGCHAIN_API_KEY": bool(os.getenv("LANGCHAIN_API_KEY")),
            "LANGCHAIN_PROJECT": bool(os.getenv("LANGCHAIN_PROJECT")),
            "LANGCHAIN_TRACING_V2": os.getenv("LANGCHAIN_TRACING_V2") == "true"
        }
        
        print("🏥 Health Check LangSmith:")
        for check, status in checks.items():
            icon = "✅" if status else "❌"
            print(f"   {icon} {check}: {status}")
        
        return all(checks.values())

# Testando a configuração
print("🔧 Testando configuração de produção...\n")

for env in ["dev", "staging", "prod"]:
    print(f"\n{'='*30} {env.upper()} {'='*30}")
    config = ConfigLangSmithProd(env)
    tags = config.configurar_tags_padrao("chat-bot-vendas", "2.1")
    print(f"🏷️  Tags sugeridas: {tags}")
    config.health_check()

print("\n💡 Use essa configuração como base para seus projetos!")

## 🎊 Resumão Final: O que Aprendemos?

**Parabéns!** 🎉 Você chegou ao final do nosso curso LangChain v0.2! 

Foram **15 módulos** de muito aprendizado, e agora você tem todo o conhecimento para construir aplicações LLM **profissionais**!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-15_img_07.png)

## 🏆 Sua Jornada Completa

### 🎯 Do Módulo 1 ao 15 - O que dominamos:

**🏗️ Fundamentos (Módulos 1-4)**
- ✅ LangChain architecture e conceitos
- ✅ ChatModels e Runnables/LCEL
- ✅ Prompt Templates e Output Parsers
- ✅ Chains e composição

**🧠 Sistemas Inteligentes (Módulos 5-8)**
- ✅ Memory Systems para contexto
- ✅ Document Loading e Splitters
- ✅ Vector Stores e Embeddings
- ✅ RAG Implementation completa

**🤖 Agentes e Projetos (Módulos 9-12)**
- ✅ Agents e Tools poderosos
- ✅ Projetos práticos completos
- ✅ Deploy com Streamlit

**🚀 Nível Avançado (Módulos 13-15)**
- ✅ LangChain v1.0 e comparações
- ✅ LangGraph para workflows complexos
- ✅ **LangSmith para observabilidade** 👈 VOCÊ ESTÁ AQUI!

### 🎯 Hoje você aprendeu:
- 🔍 **Observabilidade completa** com LangSmith
- 📊 **Métricas e monitoramento** em tempo real
- 🐛 **Debugging avançado** de chains
- 📈 **Otimização baseada em dados**
- 🚨 **Sistemas de alertas** inteligentes
- 💡 **Melhores práticas** para produção

**Dica final do Pedro**: Agora você tem todas as ferramentas para construir aplicações LLM **profissionais** e **escaláveis**! 🚀

In [ ]:
# 🎉 PARABÉNS! Você concluiu o curso!
# Vamos celebrar com um resumo visual da sua jornada

import matplotlib.pyplot as plt
import numpy as np

# Dados da jornada de aprendizado
modulos = [
    "Introdução", "ChatModel", "Prompts", "Chains", "Memory",
    "Documents", "Vectors", "RAG", "Agents", "Projeto 1",
    "Projeto 2", "Deploy", "v1.0", "LangGraph", "LangSmith"
]

# Skills acumuladas (simulação)
skills_acumuladas = np.array([5, 15, 25, 35, 45, 55, 65, 75, 85, 90, 95, 98, 100, 105, 110])

# Gráfico da evolução
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Gráfico 1: Evolução das habilidades
ax1.plot(range(len(modulos)), skills_acumuladas, 'o-', 
         linewidth=3, markersize=8, color='#FF6B6B')
ax1.fill_between(range(len(modulos)), skills_acumuladas, 
                 alpha=0.3, color='#FF6B6B')

# Destacando marcos importantes
marcos = [3, 7, 11, 14]  # Chains, RAG, Deploy, LangSmith
for marco in marcos:
    ax1.scatter(marco, skills_acumuladas[marco], s=200, 
               color='gold', edgecolor='orange', linewidth=2, zorder=5)
    ax1.annotate('🏆', xy=(marco, skills_acumuladas[marco]), 
                xytext=(0, 15), textcoords='offset points', 
                ha='center', fontsize=16)

ax1.set_title('🚀 Sua Jornada de Aprendizado - LangChain v0.2', 
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xlabel('Módulos do Curso')
ax1.set_ylabel('Nível de Habilidade')
ax1.set_xticks(range(len(modulos)))
ax1.set_xticklabels(modulos, rotation=45, ha='right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 120)

# Adicionando anotações de fases
ax1.axvspan(-0.5, 3.5, alpha=0.1, color='blue', label='🏗️ Fundamentos')
ax1.axvspan(3.5, 7.5, alpha=0.1, color='green', label='🧠 Sistemas Inteligentes')
ax1.axvspan(7.5, 11.5, alpha=0.1, color='orange', label='🤖 Projetos Práticos')
ax1.axvspan(11.5, 14.5, alpha=0.1, color='red', label='🚀 Nível Avançado')
ax1.legend(loc='upper left')

# Gráfico 2: Distribuição de conhecimentos
conhecimentos = {
    'LangChain Core': 95,
    'RAG Systems': 90,
    'Agents & Tools': 85,
    'Production Deploy': 80,
    'LangGraph': 75,
    'LangSmith': 90,
    'Best Practices': 88
}

areas = list(conhecimentos.keys())
scores = list(conhecimentos.values())
colors = plt.cm.Set3(np.linspace(0, 1, len(areas)))

bars = ax2.barh(areas, scores, color=colors)
ax2.set_title('📊 Seu Nível de Conhecimento por Área', 
              fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Nível de Proficiência (%)')
ax2.set_xlim(0, 100)

# Adicionando valores nas barras
for bar, score in zip(bars, scores):
    width = bar.get_width()
    ax2.text(width + 1, bar.get_y() + bar.get_height()/2, 
             f'{score}%', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Mensagem final
print("🎊" * 50)
print("🎉 PARABÉNS! CURSO CONCLUÍDO COM SUCESSO! 🎉")
print("🎊" * 50)
print()
print("🏆 CONQUISTAS DESBLOQUEADAS:")
print("   ✅ LangChain Expert")
print("   ✅ RAG Systems Master")
print("   ✅ Production Ready Developer")
print("   ✅ LangSmith Monitoring Pro")
print("   ✅ LangGraph Workflow Architect")
print()
print("🚀 PRÓXIMOS PASSOS:")
print("   1. Aplique o conhecimento em projetos reais")
print("   2. Compartilhe seu aprendizado com a comunidade")
print("   3. Continue explorando novas funcionalidades")
print("   4. Mentore outros desenvolvedores")
print()
print("💡 LEMBRE-SE: O aprendizado nunca para!")
print("🙋‍♂️ Pedro Guth - Obrigado por essa jornada incrível!")
print("🎊" * 50)